### Backtesting Engine

This notebook runs a historical backtest for all previously generated portfolio selections.

**Workflow:**
1.  **Setup:** Configure all paths and backtesting parameters.
2.  **Load Price Data:** Load the master historical daily prices for all tickers.
3.  **Discover Selections:** Find all selection data/parameter file pairs from the output directory.
4.  **Execute Backtests:** Iterate through each selection file, run a T+1 to T+2 simulation for each weighting scheme (EW, IV, SW), and collect performance metrics.
5.  **Save Results:** Consolidate the results from this run with any previous backtest results, removing duplicates and keeping the latest run.
6.  **Verify:** Load the master results file to confirm the process was successful.

### Setup and Configuration

This cell defines all parameters for the backtesting run, including file paths and simulation constants.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import logging
from tqdm.notebook import tqdm # For a nice progress bar

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent 
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- Local Imports ---
from config import DATE_STR, DAILY_RISK_FREE_RATE
import utils

# # --- Backtesting Parameters ---
# RISK_FREE_RATE_DAILY = 0.04 / 252

# These columns define a unique backtest. If we run a backtest for the same
# selection_date and scheme, the old result will be overwritten.
# We add strategy params to distinguish runs with different tuning.
UNIQUE_RUN_KEYS = [
    'selection_date', 'scheme', 'n_select_requested', 'inv_vol_col_name',
    'filter_min_price', 'filter_min_avg_volume_m', 'filter_min_roe_pct',
    'filter_max_debt_eq', 'score_weight_rsi', 'score_weight_change',
    'score_weight_rel_volume', 'score_weight_volatility'
]

# --- File Path Construction (using pathlib) ---
SELECTION_RESULTS_DIR = ROOT_DIR / 'output' / 'selection_results'
BACKTEST_OUTPUT_DIR = ROOT_DIR / 'output' / 'backtest_results'
HISTORICAL_PRICES_PATH = DATA_DIR / 'df_adj_close.parquet'
MASTER_RESULTS_PATH = BACKTEST_OUTPUT_DIR / 'backtest_master_results.parquet'
LOG_DIR = ROOT_DIR / 'logs'

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
%load_ext autoreload
%autoreload 2

# --- Initialize Logging ---
log_filepath = utils.setup_backtest_logging(LOG_DIR)
run_timestamp = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")

print(f"ROOT_DIR: {ROOT_DIR}")
print(f"Price Data Path: {HISTORICAL_PRICES_PATH}")
print(f"Selection Results Path: {SELECTION_RESULTS_DIR}")
print(f"Master Backtest Output Path: {MASTER_RESULTS_PATH}")

2025-12-28 20:22:37,338 - INFO - Logging initialized. Log file: c:\Users\ping\Files_win10\python\py311\stocks\logs\backtest_run_20251228_202237.log
ROOT_DIR: c:\Users\ping\Files_win10\python\py311\stocks
Price Data Path: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet
Selection Results Path: c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results
Master Backtest Output Path: c:\Users\ping\Files_win10\python\py311\stocks\output\backtest_results\backtest_master_results.parquet


### Step 1: Load Historical Price Data

In [2]:
print(f"--- Step 1: Loading historical price data ---")
df_adj_close = utils.load_price_data(HISTORICAL_PRICES_PATH)

if df_adj_close is not None:
    print("Price data loaded successfully.")
    display(df_adj_close.head(3))
    display(df_adj_close.tail(3))
else:
    print("ERROR: Halting execution as price data failed to load.")

--- Step 1: Loading historical price data ---
2025-12-28 20:22:37,640 - INFO - Loading historical price data from: c:\Users\ping\Files_win10\python\py311\stocks\data\df_adj_close.parquet
2025-12-28 20:22:38,187 - INFO - Successfully loaded and prepared price data. Shape: (250, 1548)
Price data loaded successfully.


Ticker,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ACWI,ACWX,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEIS,AEM,AEP,AER,AES,AFG,AFL,AFRM,AG,AGCO,AGG,AGI,AGNC,AHR,AIG,AIQ,AIRR,AIT,AIZ,AJG,AKAM,AL,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMG,AMGN,AMH,AMKR,AMLP,AMP,AMT,AMTM,AMX,AMZN,AN,ANET,AON,AOS,APA,APD,APG,APH,APLD,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARKK,ARM,ARMK,ARWR,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVAV,AVB,AVDE,AVDV,AVEM,AVGO,AVLV,AVTR,AVUS,AVUV,AVY,AWI,AWK,AXIA,AXON,AXP,AXS,AXSM,AXTA,AYI,AZN,AZO,B,BA,BABA,BAC,BAH,BAI,BALL,BAM,BAP,BAX,BBAX,BBCA,BBD,BBEU,BBIN,BBIO,BBJP,BBUS,BBVA,BBY,BCE,BCH,BCS,BDX,BE,BEKE,BEN,BEP,BEPC,BF-A,BF-B,BG,BHP,BIDU,BIIB,BIL,BILI,BINC,BIO,BIP,BIRK,BIV,BJ,BK,BKLC,BKLN,BKNG,BKR,BLD,BLDR,BLK,BLV,BMO,BMRN,BMY,BN,BND,BNDX,BNS,BNT,BNTX,BOKF,BOND,BOXX,BP,BPOP,BR,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSCQ,BSCR,BSV,BSX,BSY,BTC,BTI,BUD,BUFR,BURL,BVN,BWA,BWXT,BX,BXP,BYD,BZ,C,CACI,CADE,CAE,CAG,CAH,CARR,CART,CASY,CAT,CB,CBOE,CBRE,CBSH,CCEP,CCI,CCJ,CCK,CCL,CDE,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CGBL,CGCP,CGDV,CGGO,CGGR,CGMU,CGUS,CGXU,CHD,CHDN,CHKP,CHRW,CHT,CHTR,CHWY,CI,CIB,CIBR,CIEN,CIFR,CIGI,CINF,CL,CLF,CLH,CLS,CLX,CM,CMA,CMC,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLB,COO,COP,COPX,COR,COST,COWZ,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRK,CRL,CRM,CRS,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTRE,CTSH,CTVA,CUBE,CVE,CVNA,CVS,CVX,CW,CWB,CWEN,CWEN-A,CX,CYBR,CYTK,D,DAL,DASH,DAY,DB,DBEF,DBX,DCI,DD,DDOG,DDS,DE,DECK,DELL,DEO,DFAC,DFAE,DFAI,DFAS,DFAT,DFAU,DFAX,DFCF,DFEM,DFIC,DFIS,DFIV,DFLV,DFSD,DFSV,DFUS,DFUV,DG,DGRO,DGRW,DGX,DHI,DHR,DIA,DIHP,DINO,DIS,DIVO,DKNG,DKS,DLN,DLR,DLTR,DOC,DOCS,DOCU,DON,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DSI,DT,DTE,DTM,DUHP,DUK,DUOL,DVA,DVN,DVY,DXCM,DXJ,DY,DYNF,E,EA,EAGG,EBAY,EC,ECL,ED,EDU,EDV,EEM,EFA,EFAV,EFG,EFV,EFX,EG,EGO,EGP,EHC,EIX,EL,ELAN,ELS,ELV,EMA,EMB,EMBJ,EME,EMLC,EMN,EMR,EMXC,ENB,ENSG,ENTG,EOG,EPAM,EPD,EQH,EQIX,EQNR,EQR,EQT,EQX,ERIC,ERIE,ES,ESAB,ESGD,ESGE,ESGU,ESGV,ESLT,ESS,ESTC,ET,ETHA,ETN,ETR,EUFN,EVR,EVRG,EVTR,EW,EWBC,EWJ,EWT,EWY,EWZ,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,EZU,F,FAF,FANG,FAST,FBCG,FBND,FBTC,FCFS,FCNCA,FCX,FDL,FDN,FDS,FDVV,FDX,FE,FELC,FELG,FENI,FER,FERG,FEZ,FFIV,FHN,FICO,FIS,FITB,FIVE,FIX,FLEX,FLOT,FLR,FLS,FLUT,FMDE,FMS,FMX,FN,FND,FNDA,FNDE,FNDF,FNDX,FNF,FNV,FOX,FOXA,FPE,FR,FRHC,FROG,FRT,FSEC,FSLR,FSS,FSV,FTAI,FTCS,FTEC,FTI,FTNT,FTS,FTSM,FTV,FUTU,FVD,FWONA,FWONK,FXI,FYBR,G,GAP,GBIL,GBTC,GD,GDDY,GDS,GDX,GDXJ,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLD,GLDM,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOOG,GOOGL,GOVT,GPC,GPN,GRAB,GRID,GRMN,GS,GSAT,GSIE,GSK,GSLC,GTLB,GTLS,GUNR,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HBM,HCA,HD,HDB,HDV,HEFA,HEI,HEI-A,HESM,HIG,HII,HIMS,HL,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HON,HOOD,HPE,HPQ,HQY,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,HYG,IAG,IAGG,IAU,IAUM,IBB,IBIT,IBKR,IBM,IBN,IBP,ICE,ICL,ICLR,ICSH,IDA,IDCC,IDEV,IDV,IDXX,IEF,IEFA,IEI,IEMG,IESC,IEUR,IEX,IFF,IGF,IGIB,IGM,IGSB,IGV,IHG,IHI,IJH,IJJ,IJK,IJR,IJS,IJT,ILMN,IMO,INCY,INDA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IONS,IOO,IOT,IP,IQLT,IQV,IR,IREN,IRM,ISRG,ISTB,IT,ITA,ITOT,ITT,ITUB,ITW,IUSB,IUSG,IUSV,IVE,IVV,IVW,IVZ,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,IWV,IWY,IX,IXJ,IXN,IXUS,IYF,IYR,IYW,J,JAAA,JAVA,JAZZ,JBHT,JBL,JBND,JBTM,JCI,JCPB,JD,JEF,JEPI,JEPQ,JGLO,JGRO,JHG,JHMM,JHX,JIRE,JKHY,JLL,JMBS,JMST,JMUB,JNJ,JNK,JOBY,JPIE,JPM,JPST,JQUA,JXN,KB,KBWB,KDP,KEP,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KNSL,KNX,KO,KR,KRE,KRYS,KSPI,KT,KTOS,KVUE,KVYO,KWEB,KYMR,L,LAD,LAMR,LBRDA,LBRDK,LDOS,LECO,LEN,LEVI,LFUS,LH,LHX,LI,LII,LIN,LINE,LITE,LKQ,LLY,LLYVA,LLYVK,LMBS,LMT,LNC,LNG,LNT,LOGI,LOW,LPLA,LQD,LRCX,LSCC,LTM,LULU,LUMN,LUV,LVS,LW,LYB,LYFT,LYG,LYV,MA,MAA,MAGS,MANH,MAR,MAS,MASI,MBB,MBLY,MCD,MCHI,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MDY,MELI,MET,META,MFC,MFG,MGA,MGC,MGK,MGM,MGV,MHK,MIDD,MINT,MKC,MKL,MKSI,MKTX,MLI,MLM,MMC,MMM,MMYT,MNDY,MNST,MO,MOAT,MOD,MOG-A,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRUS,MRVL,MS,MSCI,MSFT,MSI,MSTR,MT,MTB,MT

Ticker,A,AA,AAL,AAON,AAPL,ABBV,ABEV,ABNB,ABT,ACGL,ACI,ACM,ACN,ACWI,ACWX,ADBE,ADC,ADI,ADM,ADP,ADSK,ADT,AEE,AEG,AEIS,AEM,AEP,AER,AES,AFG,AFL,AFRM,AG,AGCO,AGG,AGI,AGNC,AHR,AIG,AIQ,AIRR,AIT,AIZ,AJG,AKAM,AL,ALAB,ALB,ALC,ALGN,ALL,ALLE,ALLY,ALNY,ALSN,ALV,AM,AMAT,AMCR,AMD,AME,AMG,AMGN,AMH,AMKR,AMLP,AMP,AMT,AMTM,AMX,AMZN,AN,ANET,AON,AOS,APA,APD,APG,APH,APLD,APO,APP,APPF,APTV,AR,ARCC,ARE,ARES,ARGX,ARKK,ARM,ARMK,ARWR,AS,ASML,ASND,ASR,ASTS,ASX,ATI,ATO,ATR,AU,AUR,AVAV,AVB,AVDE,AVDV,AVEM,AVGO,AVLV,AVTR,AVUS,AVUV,AVY,AWI,AWK,AXIA,AXON,AXP,AXS,AXSM,AXTA,AYI,AZN,AZO,B,BA,BABA,BAC,BAH,BAI,BALL,BAM,BAP,BAX,BBAX,BBCA,BBD,BBEU,BBIN,BBIO,BBJP,BBUS,BBVA,BBY,BCE,BCH,BCS,BDX,BE,BEKE,BEN,BEP,BEPC,BF-A,BF-B,BG,BHP,BIDU,BIIB,BIL,BILI,BINC,BIO,BIP,BIRK,BIV,BJ,BK,BKLC,BKLN,BKNG,BKR,BLD,BLDR,BLK,BLV,BMO,BMRN,BMY,BN,BND,BNDX,BNS,BNT,BNTX,BOKF,BOND,BOXX,BP,BPOP,BR,BRK-A,BRK-B,BRKR,BRO,BROS,BRX,BSAC,BSBR,BSCQ,BSCR,BSV,BSX,BSY,BTC,BTI,BUD,BUFR,BURL,BVN,BWA,BWXT,BX,BXP,BYD,BZ,C,CACI,CADE,CAE,CAG,CAH,CARR,CART,CASY,CAT,CB,CBOE,CBRE,CBSH,CCEP,CCI,CCJ,CCK,CCL,CDE,CDNS,CDP,CDW,CEF,CEG,CELH,CF,CFG,CFLT,CFR,CG,CGBL,CGCP,CGDV,CGGO,CGGR,CGMU,CGUS,CGXU,CHD,CHDN,CHKP,CHRW,CHT,CHTR,CHWY,CI,CIB,CIBR,CIEN,CIFR,CIGI,CINF,CL,CLF,CLH,CLS,CLX,CM,CMA,CMC,CMCSA,CME,CMG,CMI,CMS,CNA,CNC,CNH,CNI,CNM,CNP,CNQ,COF,COHR,COIN,COKE,COLB,COO,COP,COPX,COR,COST,COWZ,CP,CPAY,CPB,CPNG,CPRT,CPT,CQP,CR,CRBG,CRDO,CRH,CRK,CRL,CRM,CRS,CRWD,CSCO,CSGP,CSL,CSX,CTAS,CTRA,CTRE,CTSH,CTVA,CUBE,CVE,CVNA,CVS,CVX,CW,CWB,CWEN,CWEN-A,CX,CYBR,CYTK,D,DAL,DASH,DAY,DB,DBEF,DBX,DCI,DD,DDOG,DDS,DE,DECK,DELL,DEO,DFAC,DFAE,DFAI,DFAS,DFAT,DFAU,DFAX,DFCF,DFEM,DFIC,DFIS,DFIV,DFLV,DFSD,DFSV,DFUS,DFUV,DG,DGRO,DGRW,DGX,DHI,DHR,DIA,DIHP,DINO,DIS,DIVO,DKNG,DKS,DLN,DLR,DLTR,DOC,DOCS,DOCU,DON,DOV,DOW,DOX,DPZ,DRI,DRS,DSGX,DSI,DT,DTE,DTM,DUHP,DUK,DUOL,DVA,DVN,DVY,DXCM,DXJ,DY,DYNF,E,EA,EAGG,EBAY,EC,ECL,ED,EDU,EDV,EEM,EFA,EFAV,EFG,EFV,EFX,EG,EGO,EGP,EHC,EIX,EL,ELAN,ELS,ELV,EMA,EMB,EMBJ,EME,EMLC,EMN,EMR,EMXC,ENB,ENSG,ENTG,EOG,EPAM,EPD,EQH,EQIX,EQNR,EQR,EQT,EQX,ERIC,ERIE,ES,ESAB,ESGD,ESGE,ESGU,ESGV,ESLT,ESS,ESTC,ET,ETHA,ETN,ETR,EUFN,EVR,EVRG,EVTR,EW,EWBC,EWJ,EWT,EWY,EWZ,EXAS,EXC,EXE,EXEL,EXLS,EXP,EXPD,EXPE,EXR,EZU,F,FAF,FANG,FAST,FBCG,FBND,FBTC,FCFS,FCNCA,FCX,FDL,FDN,FDS,FDVV,FDX,FE,FELC,FELG,FENI,FER,FERG,FEZ,FFIV,FHN,FICO,FIS,FITB,FIVE,FIX,FLEX,FLOT,FLR,FLS,FLUT,FMDE,FMS,FMX,FN,FND,FNDA,FNDE,FNDF,FNDX,FNF,FNV,FOX,FOXA,FPE,FR,FRHC,FROG,FRT,FSEC,FSLR,FSS,FSV,FTAI,FTCS,FTEC,FTI,FTNT,FTS,FTSM,FTV,FUTU,FVD,FWONA,FWONK,FXI,FYBR,G,GAP,GBIL,GBTC,GD,GDDY,GDS,GDX,GDXJ,GE,GEHC,GEN,GEV,GFI,GFL,GFS,GGAL,GGG,GH,GIB,GIL,GILD,GIS,GL,GLBE,GLD,GLDM,GLPI,GLW,GM,GMAB,GME,GMED,GNRC,GOOG,GOOGL,GOVT,GPC,GPN,GRAB,GRID,GRMN,GS,GSAT,GSIE,GSK,GSLC,GTLB,GTLS,GUNR,GWRE,GWW,H,HAL,HALO,HAS,HBAN,HBM,HCA,HD,HDB,HDV,HEFA,HEI,HEI-A,HESM,HIG,HII,HIMS,HL,HLI,HLN,HLNE,HLT,HMC,HMY,HOLX,HON,HOOD,HPE,HPQ,HQY,HRL,HSBC,HSIC,HST,HSY,HTHT,HUBB,HUBS,HUM,HWM,HYG,IAG,IAGG,IAU,IAUM,IBB,IBIT,IBKR,IBM,IBN,IBP,ICE,ICL,ICLR,ICSH,IDA,IDCC,IDEV,IDV,IDXX,IEF,IEFA,IEI,IEMG,IESC,IEUR,IEX,IFF,IGF,IGIB,IGM,IGSB,IGV,IHG,IHI,IJH,IJJ,IJK,IJR,IJS,IJT,ILMN,IMO,INCY,INDA,INFY,ING,INGR,INSM,INTC,INTU,INVH,IONQ,IONS,IOO,IOT,IP,IQLT,IQV,IR,IREN,IRM,ISRG,ISTB,IT,ITA,ITOT,ITT,ITUB,ITW,IUSB,IUSG,IUSV,IVE,IVV,IVW,IVZ,IWB,IWD,IWF,IWM,IWN,IWO,IWP,IWR,IWS,IWV,IWY,IX,IXJ,IXN,IXUS,IYF,IYR,IYW,J,JAAA,JAVA,JAZZ,JBHT,JBL,JBND,JBTM,JCI,JCPB,JD,JEF,JEPI,JEPQ,JGLO,JGRO,JHG,JHMM,JHX,JIRE,JKHY,JLL,JMBS,JMST,JMUB,JNJ,JNK,JOBY,JPIE,JPM,JPST,JQUA,JXN,KB,KBWB,KDP,KEP,KEY,KEYS,KGC,KHC,KIM,KKR,KLAC,KMB,KMI,KNSL,KNX,KO,KR,KRE,KRYS,KSPI,KT,KTOS,KVUE,KVYO,KWEB,KYMR,L,LAD,LAMR,LBRDA,LBRDK,LDOS,LECO,LEN,LEVI,LFUS,LH,LHX,LI,LII,LIN,LINE,LITE,LKQ,LLY,LLYVA,LLYVK,LMBS,LMT,LNC,LNG,LNT,LOGI,LOW,LPLA,LQD,LRCX,LSCC,LTM,LULU,LUMN,LUV,LVS,LW,LYB,LYFT,LYG,LYV,MA,MAA,MAGS,MANH,MAR,MAS,MASI,MBB,MBLY,MCD,MCHI,MCHP,MCK,MCO,MDB,MDGL,MDLZ,MDT,MDY,MELI,MET,META,MFC,MFG,MGA,MGC,MGK,MGM,MGV,MHK,MIDD,MINT,MKC,MKL,MKSI,MKTX,MLI,MLM,MMC,MMM,MMYT,MNDY,MNST,MO,MOAT,MOD,MOG-A,MOH,MORN,MOS,MPC,MPLX,MPWR,MRK,MRNA,MRUS,MRVL,MS,MSCI,MSFT,MSI,MSTR,MT,MTB,MT

### Step 2: Discover Selection Files to Backtest

In [ ]:
if df_adj_close is not None:
    print(f"\n--- Step 2: Discovering selection files in {SELECTION_RESULTS_DIR} ---")
    file_pairs = utils.find_and_pair_selection_files(SELECTION_RESULTS_DIR)

    if not file_pairs:
        print("No file pairs found to process. Halting.")
    else:
        print(f"Found {len(file_pairs)} pairs to backtest.")
        # Print the first few for inspection
        for i, (d, p) in enumerate(file_pairs[:3]):
            print(f"  Pair {i+1}: {d.name} | {p.name}")
else:
    print("Skipping file discovery.")
    file_pairs = []


--- Step 2: Discovering selection files in c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results ---
2025-12-28 20:22:41,992 - INFO - Searching for selection files in: c:\Users\ping\Files_win10\python\py311\stocks\output\selection_results
2025-12-28 20:22:42,033 - INFO - Found 133 paired data and parameter files.
Found 133 pairs to backtest.
  Pair 1: 2025-04-25_short_term_mean_reversion.parquet | 2025-04-25_short_term_mean_reversion_params.json
  Pair 2: 2025-04-28_short_term_mean_reversion.parquet | 2025-04-28_short_term_mean_reversion_params.json
  Pair 3: 2025-04-29_short_term_mean_reversion.parquet | 2025-04-29_short_term_mean_reversion_params.json


In [4]:
file_pairs[-5::]

[(WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-19_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-19_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-22_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-22_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-23_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-23_short_term_mean_reversion_params.json')),
 (WindowsPath('c:/Users/ping/Files_win10/python/py311/stocks/output/selection_results/2025-12-26_short_term_mean_reversion.parquet'),
  WindowsPath('c:/Users/ping/Files_win10/python

### Step 3: Execute Backtests

In [ ]:
all_performance_records = []
if file_pairs:
    print(
        f"\n--- Step 3: Executing backtests for {len(file_pairs)} selection dates ---"
    )

    # Use tqdm for a progress bar
    for data_file, param_file in tqdm(file_pairs, desc="Backtesting Selections"):
        records = utils.process_backtest_for_pair(
            data_file=data_file,
            param_file=param_file,
            df_adj_close=df_adj_close,
            risk_free_rate_daily=DAILY_RISK_FREE_RATE,
            run_timestamp=run_timestamp,
            log_filepath=log_filepath,
        )
        all_performance_records.extend(records)

    print(
        f"Backtesting complete. Generated {len(all_performance_records)} new performance records."
    )
else:
    print("No backtests to execute.")


--- Step 3: Executing backtests for 133 selection dates ---


Backtesting Selections:   0%|          | 0/133 [00:00<?, ?it/s]

2025-12-28 20:22:42,901 - INFO - ------------------------------
2025-12-28 20:22:42,904 - INFO - Initiating Backtest Run...
2025-12-28 20:22:42,907 - INFO -   Date          : 2025-04-25
2025-12-28 20:22:42,909 - INFO -   Scheme        : EW
2025-12-28 20:22:42,913 - INFO -   Num Tickers   : 10
2025-12-28 20:22:42,917 - INFO -   Selection Date Used: 2025-04-25
2025-12-28 20:22:42,919 - INFO -   Buy Date           : 2025-04-28
2025-12-28 20:22:42,922 - INFO -   Sell Date          : 2025-04-29
2025-12-28 20:22:42,934 - WARNING -     - Ticker SKX not in price data. Skipping.
2025-12-28 20:22:42,937 - WARNING -     - Ticker FI not in price data. Skipping.
2025-12-28 20:22:42,940 - INFO -   Trades Executed: 8/10
2025-12-28 20:22:42,942 - INFO -   Portfolio Return : 0.0033
2025-12-28 20:22:42,948 - INFO -   Win Rate         : 62.50%
2025-12-28 20:22:42,951 - INFO - Backtest simulation for 'EW' on 2025-04-25 completed.
2025-12-28 20:22:42,954 - INFO - ------------------------------
2025-12-28 2

### Step 4: Consolidate and Save Results

In [ ]:
if all_performance_records:
    print("\n--- Step 4: Consolidating and saving all results ---")
    utils.update_and_save_results(
        new_records=all_performance_records,
        results_path=MASTER_RESULTS_PATH,
        unique_key_cols=UNIQUE_RUN_KEYS,
    )
else:
    print("\nNo new results to save.")


--- Step 4: Consolidating and saving all results ---
2025-12-28 20:22:58,652 - INFO - Updating results file at: c:\Users\ping\Files_win10\python\py311\stocks\output\backtest_results\backtest_master_results.parquet
2025-12-28 20:22:58,664 - INFO - Loading backtest_master_results.parquet to merge with new results.
2025-12-28 20:22:58,757 - INFO - --------------------------------------------------
2025-12-28 20:22:58,762 - INFO - Results successfully updated and saved to 'backtest_master_results.parquet'
2025-12-28 20:22:58,764 - INFO - Records before de-duplication: 786
2025-12-28 20:22:58,765 - INFO - Duplicate records removed:      393
2025-12-28 20:22:58,767 - INFO - Final records in file:          393
2025-12-28 20:22:58,768 - INFO - --------------------------------------------------


### Step 5: Verify Saved Results

In [7]:
print("\n--- Step 5: Verifying saved master results file ---")
if MASTER_RESULTS_PATH.exists():
    df_results = pd.read_parquet(MASTER_RESULTS_PATH)
    print(f"Successfully loaded master results file with {len(df_results)} records.")
    print("Displaying head of results dataframe (latest runs):")
    display(df_results.head())
else:
    print(f"ERROR: Master results file was not found at {MASTER_RESULTS_PATH}")


--- Step 5: Verifying saved master results file ---
Successfully loaded master results file with 393 records.
Displaying head of results dataframe (latest runs):


,actual_selection_date_used,average_return,filter_max_debt_eq,filter_min_avg_volume_m,filter_min_price,filter_min_roe_pct,inv_vol_col_name,log_file,n_select_actual,n_select_requested,num_attempted_trades,num_failed_or_skipped_trades,num_selected_tickers,num_successful_trades,portfolio_return,portfolio_return_normalized,run_timestamp,scheme,score_weight_change,score_weight_rel_volume,score_weight_rsi,score_weight_volatility,selection_date,sharpe_ratio_period,std_dev_return,total_weight_traded,win_rate
0,2025-12-23,-0.0042,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20251228_202237.log,10,10,10,0,10,10,-0.0042,-0.0042,2025-12-28 20:22:37,EW,0.3500,0.2000,0.3500,0.1000,2025-12-23,-0.4566,0.0095,1.0000,0.3000
1,2025-12-23,-0.0042,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20251228_202237.log,10,10,10,0,10,10,-0.0037,-0.0037,2025-12-28 20:22:37,IV,0.3500,0.2000,0.3500,0.1000,2025-12-23,-0.4566,0.0095,1.0000,0.3000
2,2025-12-23,-0.0042,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20251228_202237.log,10,10,10,0,10,10,-0.0057,-0.0057,2025-12-28 20:22:37,SW,0.3500,0.2000,0.3500,0.1000,2025-12-23,-0.4566,0.0095,1.0000,0.3000
3,2025-12-22,0.0081,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20251228_202237.log,10,10,10,0,10,10,0.0081,0.0081,2025-12-28 20:22:37,EW,0.3500,0.2000,0.3500,0.1000,2025-12-22,0.4539,0.0176,1.0000,0.6000
4,2025-12-22,0.0081,1.5000,2.0000,10.0000,5.0000,ATR/Price %,backtest_run_20251228_202237.log,10,10,10,0,10,10,0.0087,0.0087,2025-12-28 20:22:37,IV,0.3500,0.2000,0.3500,0.1000,2025-12-22,0.4539,0.0176,1.0000,0.6000
